In [9]:
import glob
import io
import librosa
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np 
import os
import pandas as pd
import IPython.display as ipd
import seaborn as sns 
import soundfile as sf
import zipfile 

from tqdm import tqdm

In [10]:
output_path = '/Users/thomas/spé-IA/spe_ia_clics_odontocetes/.results/'

if not os.path.exists(output_path):
    os.makedirs(output_path)

    
# Set the path to the downloaded data
download_path = Path.cwd() / ".dataset"

# Read labels file
labels_file = download_path / "Y_train_ofTdMHi.csv"
df = pd.read_csv(labels_file)

# Construct file path by concatenating folder and file name
df["relative_path"] = str(download_path) + "/X_train/" + df["id"]

# Drop id column (replaced it with relative_path)
df.drop(columns=["id"], inplace=True)

df.rename(columns={"pos_label": "label"}, inplace=True)

# invert relative_path and label columns positions
df = df[["relative_path", "label"]]
display(ipd.Markdown(f"### There are **_{len(df)}_** audio files in the dataset."))

table = f"""
Here is the split into good and bad signals:
| Label   | Count   |
|:-------:|:-------:|
| 0       | {df['label'].value_counts()[0]} |
| 1       | {df['label'].value_counts()[1]} |"""
display(ipd.Markdown(table))
df.head()

### There are **_23168_** audio files in the dataset.


Here is the split into good and bad signals:
| Label   | Count   |
|:-------:|:-------:|
| 0       | 13718 |
| 1       | 9450 |

,relative_path,label
0,/Users/thomas/spé-IA/spe_ia_clics_odontocetes/...,0.0
1,/Users/thomas/spé-IA/spe_ia_clics_odontocetes/...,1.0
2,/Users/thomas/spé-IA/spe_ia_clics_odontocetes/...,1.0
3,/Users/thomas/spé-IA/spe_ia_clics_odontocetes/...,1.0
4,/Users/thomas/spé-IA/spe_ia_clics_odontocetes/...,1.0


In [34]:
def get_images(samples, sr, output_path):
    n_fft = 2048
    hop_length = 512
    n_mels=128
    S = librosa.feature.melspectrogram(y=samples,
                                       sr=sr,
                                       n_fft=n_fft,
                                       hop_length=hop_length,
                                       n_mels=n_mels,
                                       fmax=100000)
    S_db = librosa.power_to_db(S, ref=np.max)
    fig, ax = plt.subplots(figsize=(2,2))
    librosa.display.specshow(S_db,
                             x_axis='time',
                             y_axis='linear',
                             sr=sr,
                             hop_length=hop_length, 
                             )
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(output_path, dpi=150, format='png' , bbox_inches='tight', pad_inches=0)
    plt.close()

### Converting .wav to .aiff

In [35]:
file_paths = df.loc[:10, "relative_path"]
output_directory = "./aiff_files"

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

for file_path in file_paths:
    input_wav = file_path
    
    output_filename = os.path.basename(file_path).replace('.wav', '.aiff')  # Create the corresponding output file name
    output_aiff = os.path.join(output_directory, output_filename)

    data, samplerate = sf.read(input_wav)
    sf.write(output_aiff, data, samplerate, format="AIFF")

In [49]:
input_directory = Path.cwd() / ".dataset/X_train"
output_directory = "./spectogram_images"
file_limit = 10  # Number of files to process

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

file_count = 0  # Initialize file counter

for filename in os.listdir(input_directory):
    if file_count < file_limit and filename.endswith(".wav"):  # Check the file extension
        file_path = os.path.join(input_directory, filename)
        output_path = os.path.join(output_directory, f"{os.path.splitext(filename)[0]}.png")
        
        audio_data, sr = librosa.load(file_path, sr=None)
        get_images(audio_data, sr, output_path)
        
        file_count += 1  # Increment the file counter after processing a file

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/librosa/feature/spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)
